In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('AMES_TRAIN.csv')
test = pd.read_csv('AMES_TEST_SFAM.csv')
train.columns = [s.lower() for s in train.columns]
test.columns = [s.lower() for s in test.columns]
train['qualityindex'] = (train.overallqual*train.overallcond)
train['totalsqftcalc'] = (train.bsmtfinsf1+train.bsmtfinsf2+train.grlivarea)
train['pricesqfoot'] = (train['saleprice']/train['totalsqftcalc'])
test['qualityindex'] = (test.overallqual*test.overallcond)
test['totalsqftcalc'] = (test.bsmtfinsf1+test.bsmtfinsf2+test.grlivarea)
test['pricesqfoot'] = (test['saleprice']/test['totalsqftcalc'])
train=train.replace({'totalsqftcalc': {np.NaN : 10000}})
train=train.replace({'lotarea': {np.NaN : 10000}})
train=train.replace({'lotfrontage': {np.NaN : 10000}})
test=test.replace({'totalsqftcalc': {np.NaN : 10000}})
test=test.replace({'lotarea': {np.NaN : 10000}})
test=test.replace({'lotfrontage': {np.NaN : 10000}})

In [ ]:
#print(train.head())
#print(test.head())
#print(train['pricesqfoot'])
#list(train)

In [ ]:
train.plot.scatter(y='saleprice', x='totalsqftcalc') 
#This lets me see some outliars that could be distorting the data.

In [ ]:
train.plot.scatter(y='totalsqftcalc', x='saleprice') 

In [ ]:
train.plot.scatter(y='saleprice', x='lotarea') 

In [ ]:
#train.plot.bar(y='saleprice', x='zoning') 

In [ ]:
train = train[train['totalsqftcalc'] < 6000]
train = train[train['salecondition'] == 'Normal']
train = train[train['saleprice'] <= 500000]
train = train[train['lotarea'] <= 25000]
train = train[train['zoning'] != 'I']
train = train[train['zoning'] != 'C']
train = train[train['zoning'] != 'A']

In [ ]:
X = train[['saleprice','qualityindex','totalsqftcalc','yearbuilt','lotarea','lotfrontage']].copy()
X1 = train[['qualityindex','totalsqftcalc','yearbuilt','lotarea','lotfrontage']].copy()
corr = X[X.columns].corr()
corr

In [ ]:
Y = train[['saleprice']].copy()
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
select_top_3 = SelectKBest(score_func=chi2, k = 3)
fit = select_top_3.fit(X1,Y)
features = fit.transform(X1)
features[0:5] 

In [ ]:
model1 = smf.ols(formula='Y ~ qualityindex+totalsqftcalc+C(lotconfig)+C(neighborhood)+C(housestyle)+yearbuilt+C(roofstyle)+C(heating)', data=train).fit()
model1.summary()

In [ ]:
predictions_1 = model1.fittedvalues
predictions_1.head()

In [ ]:
test_predictions_1 = model1.predict(test)
d = {'p_saleprice': test_predictions_1}
df1 = test[['index']]
df2=pd.DataFrame(data=d)
Strouse_TestPredictions_1 = pd.concat([df1,df2],axis = 1, join_axes=[df1.index])
Strouse_TestPredictions_1.head()

In [ ]:
Strouse_TestPredictions_1.to_csv('logan_strouse_HW2_MODEL_1.csv')

In [ ]:
import seaborn as sns
ax = sns.boxplot(x="neighborhood", y="residual", data=n_file)
ax.set_title("Resdiual by Neighborhood")
ax.set_ylabel("Residual")
ax.set_xlabel("Neighborhood")
plt.xticks(rotation=75)
plt.show()

In [ ]:
neighborhood_predictions = model1.fittedvalues
neighborhood_predictions.head()
tr = {'p_saleprice': neighborhood_predictions}
df1 = train[['index','saleprice','neighborhood','totalsqftcalc']]
df2=pd.DataFrame(data=tr)
n_file = pd.concat([df1,df2],axis = 1, join_axes=[df1.index])

n_file['residual'] = (n_file.saleprice-n_file.p_saleprice)
n_file['actual_ppsf'] = (n_file.saleprice/n_file.totalsqftcalc)
n_file['predicted_ppsf'] = (n_file.p_saleprice/n_file.totalsqftcalc)
compare = n_file[['neighborhood','actual_ppsf','predicted_ppsf']]
compare

In [ ]:
grouping = n_file[['neighborhood','actual_ppsf','predicted_ppsf']].groupby('neighborhood').mean()
grouping_sorted = grouping.sort_values(by=['actual_ppsf'], ascending=[False])
grouping_sorted
#neighborhoods sorted by actual ppsf

In [ ]:
grouping.plot(style=['o','rx'])
# plotted actual vs. predicted against each other to see how close the dots would meet based on mean ppsf.

In [ ]:
group1 = ['GrnHill', 'Blmngtn', 'NridgHt','Somerst','StoneBr','Timber','Gilbert','CollgCr','NoRidge']
group2 = ['Crawfor','Blueste','SawyerW','Greens','BrkSide','Veenker','Mitchel','IDOTRR','OldTown']
group3 = ['ClearCr','NWAmes','NPkVill','NAmes','Sawyer','Edwards','BrDale','SWISU','MeadowV']
c_2 = compare.copy()
c_2['Neighborhood_Group'] = c_2['neighborhood'].isin(group1).astype('int64')
c_2

In [ ]:
#dir(model2)

In [ ]:
#y = np.array(train['saleprice'])
#X = train[['qualityindex','totalsqftcalc','yearbuilt','yearremodel','wooddecksf','openporchsf' ]].copy()
#X.head()
#model = feature_selection.SelectKBest(score_func=feature_selection.f_regression, k=3) # will need to convert y to an array
#results = model.fit(X,y)
#scores are the F values for each variable, bigger is better
#results.scores

In [ ]:
#from sklearn.model_selection import train_test_split
#train, test = train_test_split(train, test_size = 0.3)